In [11]:
import xml.etree.ElementTree as ET
import os
import re
from xlwt import Workbook
import xml

In [2]:
# importar el Spanish Standford POS tagger

from nltk.tag.stanford import StanfordPOSTagger
spanish_postagger = StanfordPOSTagger('standford_tagger/models/spanish.tagger', 
                                      'standford_tagger/stanford-postagger.jar')

In [14]:
class Char:
    """
    Clase definida para representar los caracteres extraídos del archivo xml
    """
    def __init__(self, text, bbox, size, font):
        self.text = text
        self.bbox = [int(float(x))//2 for x in bbox]
        self.size = size
        self.font = font
        self.area = (bbox[2]-bbox[0])*(bbox[3]-bbox[1])
        self.height = bbox[3] - bbox[1]

**Función que nos permite convertir un pdf a xml**

In [4]:
def pdf2xml(pdf_path, output_path):
    """
    Esta función realiza la conversión de un archivo pdf a un archivo xml
    
    Parámetros
    - pdf_path: str
        dirección del archivo pdf
    - output_path: str
        dirección en la que se guardará el archivo xml generado
    
    """
    comando = f"pdf2txt.py -o {output_path} -A -t xml {pdf_path}"
    result = os.system(comando)
    if result==0: 
        print("finished")
    else:
        print("error")

**Funciones para trabajar con caracteres**

In [5]:
def get_all_chars_in_page(page, debug=False, ignorar_super=False):
    """
    Esta función extrae todos los caracteres de una página (estructura de datos)
    generada por el convertidor de pdf a xml
    
    Estructura de los archivos XML
    pages->    page -> textbox -> textline -> text
                     -> rect
                     -> layout
                     
    Parámetros
    - page: 
        páginas de una resolución
    """
    total_list = []
    last_char = ""
    last_size = 0
    last_x_dim = 0
    last_y_dim = 0
    last_font = ""
    last_was_sup = False
    for textbox in page.findall('textbox'):
            for texline in textbox.findall('textline'):
                for text in texline.findall("text"):
                    if 'bbox' in text.attrib: # tag is no empty
                        # extraer el texto, la posición, tamaño y tipo de fuente
                        char = text.text
                        bbox = [int(float(y)) for y in text.attrib['bbox'].split(",")]
                        #bbox[1] = int(bbox[1]) # change this to integer
                        font = text.attrib["font"]
                        size = float(text.attrib['size'])
                        
                        if ignorar_super:
                            # identificar superscripts 
                            cond = last_size > size and char.isdigit() and last_char != " "
                            if last_was_sup and char.isdigit(): # genralmente sigue un punto o espacio
                                if debug:
                                    print("found contiguous sup", char, "after", last_char, "in page", page.attrib["id"])
                                bbox[1] = last_y_dim
                                char = "<" + char +">"# add something to identify superscripts
                                size = last_size # modificar el tamaño para que sea igual al texto
                                last_was_sup = False
                            elif cond:
                                if debug:
                                    print("found supperscript", char, "after", last_char, "in page", page.attrib["id"])
                                bbox[1] = last_y_dim
                                char = "<" + char +">"# add something to identify superscripts
                                size = last_size # modificar el tamaño para que sea igual al texto
                                last_was_sup = True
                            else:
                                last_was_sup = False
                        
                        # almacenar la información del caracter
                        character_info = Char(char, bbox, size, font)
                        total_list.append(character_info)
                        
                        last_char, last_size, last_x_dim, last_y_dim = char, size, bbox[0], bbox[1]
                        last_font = font

    return total_list

In [6]:
def ordenar_caracteres(characters):
    """
    Esta función ordena los caracteres de una lista. Esto se realiza mediante la 
    posición en el eje x y y del caracter.
    
    Parámetros
    - characters: List of Char objects
        lista de caracteres
    """
    
    characters = sorted(characters, key=lambda x: (-x.bbox[1], x.bbox[0]))
    return characters

In [7]:
def char_list_to_string(characters, ordenar=True):
    """
    Esta función permite convertir una lista de caracteres a un string. Antes de
    realizar esta conversión la lista es ordenada si ordenar=True
    
    Parámetros
    - characters: List of Char objects
    """
    if ordenar:
        characters = ordenar_caracteres(characters)

    total_string = ""
    for x in characters:
        total_string += x.text

    return total_string

In [8]:
def sep_caracteres_en_parrafos(page, dif_lim=12, lista_caracteres=True):
    """
    Esta función permite separar los caracteres de una página. Para realizar esta tarea
    se compara las diferencias verticales entre los caracteres. Cuando la diferencia entre
    dos caracteres es superior al 110% del largo vertical de los caracteres, se crea una división
    de párrafos.
    
    Parámetros
    - page: 
        página de un archivo xml
    """
    
    if lista_caracteres:
        char_list = page
        char_list = ordenar_caracteres(char_list)
    else:
        char_list = get_all_chars_in_page(page)
        char_list = ordenar_caracteres(char_list)
    
    last_x_pos = char_list[0].bbox[0]
    last_y_pos = char_list[0].bbox[1]
    diferencias = []
    heights = [x.height for x in char_list]

    for i in range(len(char_list)):
        if char_list[i].text != " " and char_list[i].text != "":
            diferencias.append(last_y_pos - char_list[i].bbox[1])
            last_y_pos = char_list[i].bbox[1]
        else:
            diferencias.append(0)
    
    # separacion de parrafos basados en el tamaño de los caracters
    separadores = []
    for d, h in zip(diferencias, heights):
        if d != 0 and d > h*1.1: 
            #print(d, h)
            separadores.append(True)
        else:
            separadores.append(False)
        
    
    posiciones = [x for (x, y) in  enumerate(separadores) if y==True]
    posiciones = [0] + posiciones + [len(diferencias)]

    lista_parrafos = []
    for i in range(len(posiciones)-1):
        #print(posiciones[i], posiciones[i+1])
        parrafo = char_list[posiciones[i]: posiciones[i+1]]
        lista_parrafos.append(parrafo)
        
    return lista_parrafos

In [9]:
def extraccion_parrafos(pages, lim_texto):
    """
    Extraer todos los párrafos de todas las páginas de un archivo xml
    
    Parámetos
    - pages: 
        página de un archivo xml
    - lim_texto: float
        límite que especifica qué caracteres van a ser eliminados. Se utiliza para
        obviar información repetitiva que se localiza en la parte superior de las 
        resoluciones
    """
    parrafos_documento = []
    for page_num in range(len(pages)):
        char_list = get_all_chars_in_page(pages[page_num])
        char_list = [x for x in char_list if x.bbox[1] <= lim_texto]
        parrafos_pagina = sep_caracteres_en_parrafos(page=char_list, lista_caracteres=True)
        parrafos_documento += parrafos_pagina
        
    return parrafos_documento

**Definición del extractor de patrones basados en expresiones regulares**

In [10]:
def extraer_patrones(list_tokens, tagger, patron_buscado):
    """
    Esta función convierte list_tokens en un string, en el que cada caracter es una clase
    que es asignada por el tagger. Finalmente, se extrae todas las coincidencias con el
    patron_buscado y se las retorna en una lisa
    
    Parámetros
    - list_tokens: List[String]
    - tagger: List[String] -> String
    - patron_buscado: regular expression
    """

    tag_string = tagger(list_tokens)
        
    # extraer todas las coincidencias con el patrón deseado
    inf_ac = 0
    lista_nombres = []
    temp_tag_string = tag_string

    while True:
        # buscar la primera coincidencia con el patrón
        #resultado = re.search(noun_phrase, temp_tag_string)
        resultado = re.search(patron_buscado, temp_tag_string)
        if resultado is None: 
            break

        # extracción de la frase nominal
        inf, sup = resultado.span(0)
        nombre_encontrado = list_tokens[inf_ac+inf:inf_ac+sup]
        nombre_encontrado = " ".join(nombre_encontrado)
        lista_nombres.append(nombre_encontrado)

        # modifcar la lista de palabras para buscar la siguiente frase nominal
        inf_ac += sup
        temp_tag_string = temp_tag_string[sup:]    
        
    return lista_nombres

**Definición de los taggers**

In [11]:
def POS_tag(lista_tokens):
    """
    Esta función permite extraer las categorías gramaticales de las palabras
    documentación del Standford Spanish tagger:
        https://nlp.stanford.edu/software/spanish-faq.html
        
    Parámetros
    - lista_tokens: List[String]
        lista de palabras
    """
    tags = spanish_postagger.tag(lista_tokens) 
    return tags


def tagger_extr_leyes(tokens):
    """
    Esta función realiza la clasificación descrita en el paper.
    
    Categoría                           Clase  Ejemplo 
    Signos de puntuación                f      “,”, “.”, “- “, “(“, “)” 
    Numeral                             z      128, 12°, 12.2 
    Palabra que empieza por mayúscula   M      Ley, Código, etc. 
    Preposiciones                       s      en, de, etc. 
    Determinantes                       d      el, la, los, la, etc. 
    Conjunciones                        c      y, o, pero, etc. 
    Pronombres                          p      la, lo, las, etc. 
    Palabra que empieza por minúscula   m      requisitos, criterios, etc.
    
    Parámetros
    - tokens: List[String]

    
    """
    tagged_tokens = POS_tag(tokens)
    tagged_tokens = [(x, y[0]) for (x, y) in tagged_tokens]
    
    simpl_tags = []
    # no reconoce algunos números
    for x in tagged_tokens:
        
        # primero marcar los sígnos de puntuacion
        if x[1] in "f":
            simpl_tags.append((x[0], x[1]))
            
        # dígitos
        elif re.match("\d[\d\.º°]*", x[0]) is not None:
            simpl_tags.append((x[0], "z"))
        
        # "La" debería contar como mayúscula, 
        elif x[0].capitalize() == x[0] or x[0].upper() == x[0]:
            simpl_tags.append((x[0].capitalize(), "M"))
        
        # preposiciones, determinantes, conjunciones, artículos
        elif x[1] in "sdpc": 
            simpl_tags.append((x[0], x[1]))
            
        #   
        elif x[1] in "w":
            simpl_tags.append((x[0].capitalize(), "M"))
        
        else: 
            simpl_tags.append((x[0], "m"))
            
    tag_string = ""
    for x in simpl_tags:
        tag_string += x[1]
    
    return tag_string

**Función de pre-procesamiento de texto previo a la extraccion de frases nominales**

In [15]:
PAT_PUNT = re.compile(r'(?P<prev_w>[a-zA-Z]+)(?P<punct>[\.,:;"\'])\s+')

PAT_PUNT_N = re.compile(r'\s+(?P<prev_w>[\d]+)(?P<punct>[\.,:;"\'])\s+')

PAT_PUNT_PAR = re.compile(r'\((?P<prev_w>[\w\s,-]+)\)')

# el artículo 56, establece -> el artículo 56 , establece
SEP_DIG_PUNT = re.compile(r'(?P<prev_w>[\d]+)(?P<punct>[,:])\s+')

# artículo 5.- Se refiere -> artículo 5 .- Se refiere
SEP_DIG_PUNT_2 = re.compile(r'(?P<prev_d>[\d°º]+)(?P<punct>\.-)\s+')

def pre_procesamiento_extraccion_leyes(texto_parrafo):
    """
    Esta función realiza tareas de pre-procesamiento de texto, previas a la 
    classificación en clases, por ejemplo, separar los signos de puntuación, 
    capitalizar ciertas palabras, etc.
    
    Parámetros
    - texto_parrafo: string
    """
    
    texto_parrafo = re.sub("<\d>", "", texto_parrafo)
    texto_parrafo = re.sub("artículo", "Artículo", texto_parrafo)
    texto_parrafo = re.sub("articulo", "Artículo", texto_parrafo)
    texto_parrafo = re.sub("literal", "Literal", texto_parrafo)
    texto_parrafo = re.sub("en\s+adelante", "En Adelante", texto_parrafo)
    texto_parrafo = re.sub("modificada", "Modificada", texto_parrafo)

    # separar la puntuación
    texto_parrafo = PAT_PUNT_PAR.sub(r' ( \g<prev_w> ) ', texto_parrafo)
    texto_parrafo = PAT_PUNT.sub(r' \g<prev_w> \g<punct> ', texto_parrafo)
    texto_parrafo = PAT_PUNT_N.sub(r' \g<prev_w> \g<punct> ', texto_parrafo)
    texto_parrafo = SEP_DIG_PUNT.sub(r' \g<prev_w> \g<punct> ', texto_parrafo)
    texto_parrafo = SEP_DIG_PUNT_2.sub(r' \g<prev_d> \g<punct> ', texto_parrafo)
    return texto_parrafo

**Función de extraccion de leyes en un texto**

In [17]:
def extraer_leyes(parrafo):        
    texto_parrafo = char_list_to_string(parrafo)
    texto_parrafo = pre_procesamiento_extraccion_leyes(texto_parrafo)
    lista_tokens = re.split("\s+", texto_parrafo) # tokenizar la cadena de texto
    lista_tokens = [x for x in lista_tokens if x != ""] # filtrar tokens vacíos
    
    # extraer todas las coincidencias con el patrón especificado
    #fn_lista = extraer_patrones(lista_tokens, tagger_extr_leyes, "M[Msdcfpz]*[Mz]")
    fn_lista = extraer_patrones(lista_tokens, tagger_extr_leyes, "M[Msdcfpz]*[Mzf]")

    return  fn_lista


def extraccion_leyes_parrafos(lista_parrafos):
    lista_FN = []
    for parrafo in lista_parrafos:
        lista_FN_parr = extraer_leyes(parrafo)
        lista_FN_cap = filtrar_frases_nominales_legales(lista_FN_parr, lista_term_leg_cap)    
        lista_FN_mayus = filtrar_frases_nominales_legales(lista_FN_parr, lista_term_leg_upper)
        lista_FN += lista_FN_cap + lista_FN_mayus

    lista_FN_no_dup = []
    for x in lista_FN:
        if x not in lista_FN_no_dup: lista_FN_no_dup.append(x)
            
    return lista_FN_no_dup

In [18]:
lista_term_leg = ['decreto', 'ley', 'resolución', 'código', 'constitución', 
            'ds', 'directiva', 'sentencia', 'acuerdo', 'decisión', 'reglamento',
            'reglamento',  'resolucion', 'cláusulas', 'artículo',
            'convenio', 'texto', 'sala constitucional', 'precedente', 'codigo']

# no debe ir la resolucion
lista_term_leg_no_res = ['acuerdo sobre', 'codigo', 'código', 'cláusulas generales',
                         'decreto', 'ds', 'ley', 'constitución', 
            'directiva', 'sentencia', 'acuerdo sobre', 'decisión', 'reglamento',
            'reglamento', 'artículo',
            'convenio', 'texto único', 'sala constitucional', 'precedente obligatorio']


# no debe ir resolucion
lista_term_leg_cap = ['Acuerdo', 'Codigo', 'Código', 'Cláusulas Generales', 'Convenio',
                         'Decreto', 'Directiva', 'Sentencia', 'Ley', 'Constitución', 'Reglamento',
                         'Texto Único', 'Sala Constitucional']

lista_term_leg_upper = [x.upper() for x in lista_term_leg_cap]

def filtrar_frases_nominales_legales(lista, frases_filtro):
    """
    Dada una lista de frases nominales, se extrae aquellas que contienen por 
    lo menos una palabra incluida en frases_filtro
    """
    lista = [x for x in lista if any([(y in x) for y in frases_filtro])]
    return lista

**Funciones de procesamiento del archivos xml**

In [19]:
# 1.- hallar la posición de inicio del texto
def hallar_separador_texto(page):
    lista_char = get_all_chars_in_page(page)
    posicion_p = None
    for i in range(len(lista_char)):
        caracteres = [x.text for x in lista_char[i: i+11]]
        caracteres = "".join(caracteres)

        if caracteres.lower() == "procedencia": 
            posicion_p = i
            break

    lim_texto = lista_char[posicion_p].bbox[3]
    return lim_texto

In [20]:
def extraccion_codigo_resolucion(page, lim_texto):
    cabecera = [ x for x in get_all_chars_in_page(page) if x.bbox[3] > lim_texto]
    cabecera = char_list_to_string(cabecera).strip()
    cabecera = cabecera.replace("Nº", "")
    cabecera = cabecera.replace("N°", "")
    cabecera = cabecera.replace("­", "-")
    
    #codigo_resolucion = re.findall("RESOLUCI[OÓ]N\s+N.\s+[\w\d–/-]+", cabecera)
    codigo_resolucion = re.findall("RESOLUCI[OÓ]N\s+[A-Z\d–/-]+", cabecera)
    codigo_resolucion = codigo_resolucion[0]
    return codigo_resolucion

In [21]:
def get_summary_information(info):
    keys = re.findall("[A-ZÁÉÍÚÓ]+\s*:", info)
    keys = [x.replace(":", "").strip() for x in keys]
    
    values = re.split("[A-ZÁÉÍÚÓ]+\s*:", info)
    values = [x for x in values if x!=""]
    values = [x.strip() for x in values]
    
    assert keys != [] and values != [], "error en la extracción"
    
    # eliminar la fecha de la sanción
    patron_fecha="\w+,\s+\d+\s+\w+\s+\w+\s+\w+\s+\d+"
    result = re.findall(patron_fecha, values[-1])
    
    if result:
        values[-1] = values[-1].replace(result[0], "")
        keys.append("FECHA")
        values.append(result[0])
    
    dic = {k:v for k, v in zip(keys, values)}
    
    return dic

def extraccion_informacion_del_caso(texto):
    patron_info_ext = "\s*(PROCEDENCIA.*)\s*(I?.?\s*ANTECEDENTES.*$)"  # puede aparecer en varias paginas
    try:
        m = re.search(patron_info_ext, texto, re.DOTALL)
        info = m.group(1).strip()
    except:
        info = "X: x"
        print("Error al extraer la información")
        
    info_dic = get_summary_information(info)
    return info_dic

In [22]:
def extraccion_vocales(texto_parrafos_completo, lista_string=False):
    if lista_string:
        for x in texto_parrafos_completo:
            if "vocales" in x:
                texto_vocales = x
    else:
        c = 0
        for parrafo in texto_parrafos_completo:
            parrafo_texto = char_list_to_string(parrafo)
            if "vocales" in parrafo_texto.lower():
                break
            c+=1
        texto_vocales = char_list_to_string(texto_parrafos_completo[c])
    
    
    texto_vocales.index("vocales")

    num_y = texto_vocales.find(" y ")
    
    vocales_nombres = re.split("vocales", texto_vocales)[-1]
    vocales_nombres = vocales_nombres.replace("\xa0", " ")

    if len(re.findall(" y ", vocales_nombres)) == 1:
        vocales_nombres =  re.sub(" y ", ", ", vocales_nombres)
        vocales_nombres = re.split(",", vocales_nombres)
    else:
        vocales_nombres =  re.sub(" y ", ", ", vocales_nombres)
        vocales_nombres = re.split(",", vocales_nombres)
        #print("error al extraer los vocales")
        
    vocales_nombres = [x.strip() for x in vocales_nombres]
    
    return vocales_nombres

In [23]:
def extraccion_presidente(texto_parrafos_completo, lista_string=False):
    texto_presidente = ""
    if lista_string:
        for x in texto_parrafos_completo:
            if "presidente" in x.lower():
                texto_presidente = x
        
        presidente_nombre = ""
        re.match("[A-Z\s]+", texto_presidente)
        for x in texto_presidente:
            if x == x.upper(): presidente_nombre += x
        return presidente_nombre
    else:
        c = 0
        # hay que ubicar el último párrafo que mencione "presidente"
        for i in range(len(texto_parrafos_completo)):
            parrafo_texto = char_list_to_string(texto_parrafos_completo[i])
            if "president" in parrafo_texto.lower():
                c=i
        texto_presidente = char_list_to_string(texto_parrafos_completo[c])
        texto_presidente = texto_presidente.replace("\xa0", " ")
        texto_presidente = re.sub("[Vice]*[Pp]resident[ea]", "", texto_presidente)
        return texto_presidente.strip()

**Ejemplo del procesamiento de una resolución**

In [24]:
directory = "D:/UP/Juez-Bot/codigo/dataset/archivos_xml/2020"
lista_pdfs = os.listdir(directory)
for x in lista_pdfs: print(x)

doc_202004281334177126.xml
doc_202004281453474935.xml
doc_202008302359116266.xml
doc_202008302359126445.xml
doc_202010042345429599.xml
doc_202010131837325071.xml
doc_202010211736309593.xml


In [25]:
file_name = "doc_202004281334177126"
xml_path = f"D:/UP/Juez-Bot/codigo/dataset/archivos_xml/2020/{file_name}.xml"
xls_path = f"D:/UP/Juez-Bot/codigo/dataset/resoluciones_procesadas/2020/{file_name}"

tree = ET.parse(xml_path)
pages = tree.getroot()

In [26]:
# HALLAR EL LIMITE DEL TEXTO
lim_texto = hallar_separador_texto(pages[0])

# EXTRACCION DEL CODIGO DE RESOLUCION
codigo_resolucion = extraccion_codigo_resolucion(pages[0], lim_texto)

print(codigo_resolucion)

RESOLUCIÓN 0002-2020/SPC-INDECOPI


In [27]:
# EXTRACCION DE LOS PÁRRAFOS DEL TEXTO (SIN SEPARACION DE TEXTO PRINCIPAL Y NOTAS)
parrafos_documento = extraccion_parrafos(pages, lim_texto)

for x in parrafos_documento:
    print(char_list_to_string(x), "\n---") 

PROCEDENCIA :   COMISIÓN DE LA OFICINA REGIONAL DEL INDECOPI DE JUNÍN PROCEDIMIENTO :  DE PARTE DENUNCIANTE : JULIA CASTELLANOS VILCARANO DENUNCIADA : EMPRESA DE TRANSPORTES TURISMO SEÑOR DE ATACO S.A.C. MATERIA : DEBER DE IDONEIDAD ACTIVIDAD : OTRAS ACTIVIDADES DE TRANSPORTE POR VÍA TERRESTRE   
---
SUMILLA: Se confirma la resolución apelada, en el extremo que declaró infundada la denuncia interpuesta por la señora Julia Castellanos Vilcarano contra Empresa de Transportes Turismo Señor de Ataco S.A.C., por presunta infracción de los artículos 18° y 19° del Código de Protección y Defensa del Consumidor; al no haberse demostrado que la denunciante haya entregado       –para su custodia y traslado– el equipaje presuntamente extraviado por la denunciada, durante la prestación del servicio de transporte terrestre en la ruta Lima – Huancayo; destino al cual arribó el 26 de octubre de 2018.   
---
Lima, 3 de enero de 2020   
---
ANTECEDENTES  1. El 9 de noviembre de 2018, la señora Julia Cas

In [28]:
# EXTRAER LA INFORMACIÓN DEL CASO
total_text = " ".join([char_list_to_string(x) for x in  parrafos_documento])
info_dic = extraccion_informacion_del_caso(total_text)

if info_dic:
    for key, value in info_dic.items():
        print(f"{key}: {value}")

PROCEDENCIA: COMISIÓN DE LA OFICINA REGIONAL DEL INDECOPI DE JUNÍN
PROCEDIMIENTO: DE PARTE
DENUNCIANTE: JULIA CASTELLANOS VILCARANO
DENUNCIADA: EMPRESA DE TRANSPORTES TURISMO SEÑOR DE ATACO S.A.C.
MATERIA: DEBER DE IDONEIDAD
ACTIVIDAD: OTRAS ACTIVIDADES DE TRANSPORTE POR VÍA TERRESTRE
SUMILLA: Se confirma la resolución apelada, en el extremo que declaró infundada la denuncia interpuesta por la señora Julia Castellanos Vilcarano contra Empresa de Transportes Turismo Señor de Ataco S.A.C., por presunta infracción de los artículos 18° y 19° del Código de Protección y Defensa del Consumidor; al no haberse demostrado que la denunciante haya entregado       –para su custodia y traslado– el equipaje presuntamente extraviado por la denunciada, durante la prestación del servicio de transporte terrestre en la ruta Lima – Huancayo; destino al cual arribó el 26 de octubre de 2018.   
FECHA: Lima, 3 de enero de 2020


In [29]:
# EXTRAER LAS LEYES DEL CASO
lista_leyes = extraccion_leyes_parrafos(parrafos_documento)
lista_leyes = sorted(lista_leyes)
lista_leyes

['Artículo 173° del Texto Único Ordenado de la Ley 27444 , Ley del Procedimiento Administrativo General ,',
 'Artículo 196° del Código Procesal Civil ,',
 'Artículos 18° y 19° del Código de Protección y Defensa del Consumidor ;',
 'Asimismo , el Artículo 19º del Códigoestablece',
 'CÓDIGO PROCESAL CIVIL . Artículo 196° .- Carga',
 'CÓDIGO PROCESAL CIVIL . Disposiciones Complementarias . Disposiciones Finales . Primera . Las',
 'Decreto Supremo 004-2019-JUS ( En Adelante , el TUO de la LPAG ) ,',
 'El Artículo 18º del Código',
 'LEY 29571 . CÓDIGO DE PROTECCIÓN Y DEFENSA DEL CONSUMIDOR . Artículo 18º .- Idoneidad . Se',
 'LEY 29571 . CÓDIGO DE PROTECCIÓN Y DEFENSA DEL CONSUMIDOR . Artículo 19º .- Obligación',
 'LEY 29571 . CÓDIGO DE PROTECCIÓN Y DEFENSA DEL CONSUMIDOR . Artículo 20° .- Garantí',
 'LEY 29571 . CÓDIGO DE PROTECCIÓN Y DEFENSA DEL CONSUMIDOR Artículo 104°. - Responsabilidad',
 'Ley 29571 , Código de Protección y Defensa del Consumidor ( En Adelante , el Código ) ,',
 'TEXTO

In [30]:
extraccion_vocales(parrafos_documento, lista_string=False)

['Javier Eduardo Raymundo Villa García Vargas',
 'Juan Alejandro Espinoza Espinoza',
 'Roxana María Irma Barrantes Cáceres',
 'José Francisco Martín Perla Anaya.']

In [31]:
extraccion_presidente(parrafos_documento, lista_string=False)

'JAVIER EDUARDO RAYMUNDO VILLA GARCÍA VARGAS'

**Funcion completa de procesamiento de documentos**

In [32]:
def generar_resumen_resolucion_simplificado(input_file, excel_path, file_name, xml_file=False,
                              sep_linea_tp=15, sep_linea_nt=12, ignorar_super=False):

    if xml_file: # si ya es un archivo xml, solo abrirlo
        tree = ET.parse(input_file)
        pages = tree.getroot()
    else: # convertir el pdf a xml y abrirlo
        output_file_path = "D:/UP/Juez-Bot/codigo/output.xml"
        pdf2xml(input_file, output_file_path)
        tree = ET.parse(output_file_path)
        pages = tree.getroot()
    
    # HALLAR EL LIMITE DEL TEXTO
    lim_texto = hallar_separador_texto(pages[0])
    
    # EXTRAER LA CABECERA DEL DOCUMENTO
    codigo_resolucion = extraccion_codigo_resolucion(pages[0], lim_texto)
    
    # EXTRAER TODOS LOS PÁRRAFOS DEL DOCUMENTO
    parrafos_documento = extraccion_parrafos(pages, lim_texto)

    # EXTAER LA INFORMACIÓN DEL CASO
    total_text = " ".join([char_list_to_string(x) for x in  parrafos_documento])
    info_dic = extraccion_informacion_del_caso(total_text)
    
    # EXTRAER LEYES
    lista_leyes = extraccion_leyes_parrafos(parrafos_documento)
    lista_leyes = sorted(lista_leyes)
    
    # EXTRACCION DE LOS VOCALES Y PRESIDENTE
    try:
        vocales_nombres = extraccion_vocales(parrafos_documento, lista_string=False)
        presidente_nombre = extraccion_presidente(parrafos_documento, lista_string=False)
    except:
        vocales_nombres = []
        presidente_nombre = []
    
    # ESCRIBIR LOS RESULTADOS EN UN ARCHIVO DE EXCEL
    wb = Workbook() 
    sheet1 = wb.add_sheet('Sheet 1') 

    sheet1.write(0, 0, 'Año') 
    sheet1.write(0, 1, '2020') 

    sheet1.write(1, 0, 'Categoría')
    sheet1.write(1, 1, 'Pérdida de equipaje')

    sheet1.write(2, 0, 'Materia')
    sheet1.write(2, 1, 'Idoneidad del servicio') 

    sheet1.write(3, 0, 'Número de resolución')
    sheet1.write(3, 1, codigo_resolucion)

    sheet1.write(4, 0, 'Empresa denunciada')
    sheet1.write(4, 1, info_dic.get("DENUNCIADO", info_dic.get("DENUNCIADA", ""))) 

    sheet1.write(5, 0, 'Criterios aplicables')
    sheet1.write(6, 0, 'Medios probatorios Cargo')
    sheet1.write(7, 0, 'Medios probatorios Descargo')
    sheet1.write(8, 0, 'Cuestiones')
    
    c = 9
    sheet1.write(c, 0, 'Normativa sectorial aplicada')
    for x in lista_leyes:
        sheet1.write(c, 1, x)
        c+=1

    sheet1.write(c, 0, 'Multas')
    sheet1.write(c, 1, info_dic.get("SANCIÓN", info_dic.get("SANCIONES", "")))
    c+=1

    sheet1.write(c, 0, 'Configuracion del tribunal')
    for i in range(len(vocales_nombres)):
        sheet1.write(c, 1, vocales_nombres[i])
        c += 1
        
    sheet1.write(c, 0, 'Presidente')
    sheet1.write(c, 1, presidente_nombre)

    c += 1
    sheet1.write(c, 0, 'Decisión')

    wb.save(f"{excel_path}/{file_name}.xls") 

**Convertir archivos pdf a xml**

**Procesar archivos xml y guardarlos como xls**

In [33]:
# Leer lo archivos xml, extraer la información y guardarlos en xls
directory = "D:/UP/Juez-Bot/codigo/dataset/archivos_xml/2020"
out_path = "D:/UP/Juez-Bot/codigo/dataset/resoluciones_procesadas/2020"

lista_pdfs = os.listdir(directory)

# resolcuiones con errores
lista_pdfs = lista_pdfs[:1]
errores = {}

for file in lista_pdfs:
    print(f"Processing file {file}")
    xml_direction = f"{directory}/{file}"
    out_file_name = file.replace(".xml", "")
    
    try:
        generar_resumen_resolucion_simplificado(xml_direction, 
                                               out_path, 
                                               out_file_name,
                                               xml_file=True,
                                               sep_linea_tp=15,
                                               sep_linea_nt=13)
    except Error as e: 
        errores[file] = e

Processing file doc_202004281334177126.xml


In [34]:
# reportar los errores en el procesamiento
for key, error in errores.items():
    print(f"{key}: {error}")